# 1. 開啟瀏覽器，進到蝦皮搜尋首頁，並輸入關鍵字

## 1.1 引用套件，並準備一些瀏覽器基礎設定

In [107]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import re
from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd


# 關閉通知
options = webdriver.ChromeOptions()
prefs = {
    'profile.default_content_setting_values':
        {
            'notifications': 2
        }
}
options.add_experimental_option('prefs', prefs)
options.add_argument("disable-infobars")

## 1.2 開啟瀏覽器，在蝦皮以關鍵字進行搜尋 

In [108]:
# 打啟動selenium 務必確認driver 檔案跟python 檔案要在同個資料夾中
# 在這邊下載: https://chromedriver.chromium.org/downloads
driver = webdriver.Chrome(options=options)
keyword = "三多葉黃素"
driver.get(f"https://shopee.tw/search?keyword={keyword}&order=asc&page=0&sortBy=price")
time.sleep(4)

## 1.3 往下滑三次，確保60筆搜尋結果都有顯現出來 (效果意外地不理想，待解決)

- 好像滑過頭，沒有觸發到讀取資料

In [58]:
# 往下滑
for i in range(0,3):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(3)
print("滑好了")

滑好了


# 2. 擷取網頁資料

## 2.1 擷取並解析 HTML

In [109]:
html = etree.HTML(driver.page_source)
soup = BeautifulSoup(driver.page_source, "html.parser")

## 2.2 選擇搜尋結果

In [110]:
all_ = soup.find_all("div", class_="shopee-search-item-result")
print(len(all_))

1


In [111]:
all_total = all_[0].find_all("span", class_="shopee-mini-page-controller__total")
total = all_total[0].text
print(total)

all_current = all_[0].find_all("span", class_="shopee-mini-page-controller__current")
current = all_current[0].text
print(current)

9
1


## 2.3 獲取商品資訊

### 2.3.1 確認商品數量

In [112]:
# all_result = all_[0].find_all("div", class_="VTjd7p whIxGK")
all_result = all_[0].find_all("div", class_="col-xs-2-4 shopee-search-item-result__item")
print(len(all_result))

60


### 2.3.2 指定其中一個產品，測試抓取資料是否成功

In [113]:
# 指定要分析第幾個產品
product = all_result[10]

name = product.find_all("div", class_="ie3A+n bM+7UW Cve6sh")[0].text
price = product.find_all("div", {"class": ["vioxXd", "rVLWG6h"]})[0].text
sold = product.find_all("div", {"class": ["r6HknA", "uEPGHT"]})[0].text
img = product.find_all("img", {"class": ["_7DTxhh", "vc8g9F"]})[0]["src"]
href = product.find_all("a", {"data-sqe": "link"})[0]["href"]

if sold != "":
    sold = sold[4:]   # 移除「已售出」

print(name)
print(price)
print(sold)
print(img)
print(f"https://shopee.tw{href}")

三多 金盞花萃取物(含葉黃素) 複方軟膠囊 100粒(全新公司貨完整包裝不割條碼)
$575
310
https://cf.shopee.tw/file/fa2db0c4be6d4851f6afc24bfc6509c1_tn
httpS://shopee.tw/三多-金盞花萃取物(含葉黃素)-複方軟膠囊-100粒(全新公司貨完整包裝不割條碼)-i.126002119.4054051189?sp_atk=0039b09b-c538-4301-856a-4cd0f68ceb2b&xptdk=0039b09b-c538-4301-856a-4cd0f68ceb2b


### 2.3.3 抓取這一頁全部的產品資料，並存成 excel 和 html 檔

#### 優先處理：
    - 自動往下滑的部分要解決
    - 改用 Colab
        - 目前問題：可以用selenium得到資料，但沒有視窗開啟，且因為沒有往下滑，資料並不完整
        - 神奇的是，有60筆資料
        - 應該是中後半段有些標籤內容抓不到

#### TODO list
    - 價格低於某個數值，將表格填色
    - 如果是廣告，標註起來
    - 價格呈現優化

#### 無關緊要的TODO list
    - find_all 應該可以換成 find
    - 過長標題自動換行 (e.g. 用<br>)
    - 關鍵字 highlight (e.g. 用<mark></mark>)
    


In [117]:
def path_to_image_html(path):
    return '<img src="'+ path + '" width="100" >'

def href_to_full_path(href):
    return f'<a href="https://shopee.tw{href}">link</a>'

name_list = []
img_list = []
price_list = []
sold_list = []
href_list = []

for product in all_result:
    name = product.find_all("div", class_="ie3A+n bM+7UW Cve6sh")[0].text
#     name = product.find_all("div", {"class": ["ie3A+n", "bM+7UW" ,"Cve6sh"]})[0].text
#     name = product.find_all("div", {"class": ["ie3A+n", "bM+7UW" ,"Cve6sh"]})
    img = product.find_all("img", {"class": ["_7DTxhh", "vc8g9F"]})[0]["src"]
    price = product.find_all("div", {"class": ["vioxXd", "rVLWG6h"]})[0].text
    sold = product.find_all("div", {"class": ["r6HknA", "uEPGHT"]})[0].text
    href = product.find_all("a", {"data-sqe": "link"})[0]["href"]
    
    
    if sold != "":
        sold = sold[4:]   # 移除「已售出」
    
    name_list.append(name)
    img_list.append(path_to_image_html(img))
    price_list.append(price)
    sold_list.append(sold)
    href_list.append(href_to_full_path(href))

data = {
    "name": name_list,
    "img": img_list,
    "price": price_list,
    "sold": sold_list,
    "href": href_list
}

df = pd.DataFrame(data)

df.to_excel("data.xlsx")

# df.to_html("data.html")
df.to_html("data.html", escape=False, formatters=dict(Country=path_to_image_html))

df

,name,img,price,sold,href
0,三多 SENTOSA 金盞花葉黃素複方軟膠囊 50粒/100粒 葉黃素凍 另有素食葉黃素 ...,"<img src=""https://cf.shopee.tw/file/0c5fc34dc3...",$259 - $679,756,"<a href=""https://shopee.tw/三多-SENTOSA-金盞花葉黃素複方..."
1,三多 金盞花葉黃素複方軟膠囊【醫妝世家 2號館 x 大生鮮】金盞花 葉黃素 蝦紅素 素食金盞...,"<img src=""https://cf.shopee.tw/file/509d49f621...",$259 - $669,101,"<a href=""https://shopee.tw/三多-金盞花葉黃素複方軟膠囊【醫妝世家..."
2,三多金盞花萃取物葉黃素一盒50&100顆,"<img src=""https://cf.shopee.tw/file/34e710a94f...",$280 - $500,222,"<a href=""https://shopee.tw/三多金盞花萃取物葉黃素一盒50-100..."
3,三多 金盞花葉黃素複方軟膠囊 (50粒/100粒)【醫妝世家】葉黃素,"<img src=""https://cf.shopee.tw/file/48502a2e56...","$339 - $1,350","1,907","<a href=""https://shopee.tw/三多-金盞花葉黃素複方軟膠囊-(50粒..."
4,原廠貨 限時優惠★4盒 宅配 免運費 ★ 貨到付款~ 三多 金盞花萃取物 葉黃素 複方軟膠...,"<img src=""https://cf.shopee.tw/file/49364224bc...",$360 - $575,"2,019","<a href=""https://shopee.tw/原廠貨-限時優惠★4盒-宅配-免運費-..."
5,三多 SENTOSA 葉黃素 金盞花萃取物 (最新效期 實體店面公司貨),"<img src=""https://cf.shopee.tw/file/faadb26a7c...",$365 - $649,531,"<a href=""https://shopee.tw/三多-SENTOSA-葉黃素-金盞花萃..."
6,三多 金盞花萃取物 含葉黃素 複方軟膠囊 50粒/100粒/盒 大樹,"<img src=""https://cf.shopee.tw/file/0f5712fe24...",$420 - $798,"2,758","<a href=""https://shopee.tw/三多-金盞花萃取物-含葉黃素-複方軟膠..."
7,SENTOSA三多 金盞花葉黃素複方軟膠囊（50粒）【康是美】,"<img src=""https://cf.shopee.tw/file/fbfe917f65...",$499,"1,318","<a href=""https://shopee.tw/SENTOSA三多-金盞花葉黃素複方軟..."
8,保證原廠貨 限時優惠 2盒免運 ★ 免運費 ★ 三多 金盞花萃取物 (含葉黃素) 複方軟膠囊...,"<img src=""https://cf.shopee.tw/file/123adb7321...",$480 - $595,511,"<a href=""https://shopee.tw/保證原廠貨-限時優惠-2盒免運-★-免..."
9,(正品)三多 金盞花萃取物（含葉黃素）複方軟膠囊,"<img src=""https://cf.shopee.tw/file/daa20a399a...",$639,784,"<a href=""https://shopee.tw/(正品)三多-金盞花萃取物（含葉黃素）..."
